In [ ]:
from sympy.solvers.pde import classify_pde, pdsolve
import sympy as sp
import matplotlib
import numpy as np
import pandas as pd
from typing import Callable
import matplotlib.pyplot as plt

## Parameters

In [ ]:
N = 1200
time = [0, 240]


mu_r, mu_h, mu_m = 584, 0.26, 0
sigma_r, sigma_h, sigma_m = 197, 0.14, 0
gamma = 3.5
p = 2.47*1e6
delta = 0.03
xi = 0.02

K_r, K_h, K_m = 1166*1e6,0.39*1e6, 656*1e6
OS_r, OS_h, OS_m = 0*1e6, 0.29*1e6, 1.08*1e6
lambda_r, lambda_h, lambda_m = 0, 0, 0

## Algorithm


In [ ]:
t = np.linspace(*time, num=N)
dt = t[1] - t[0]

theta_r = np.zeros(N+1)
theta_h = np.zeros(N+1)
theta_m = np.zeros(N+1)
x = np.zeros(N+1)

S_r, S_h, S_m, W = np.zeros(N+1), np.zeros(N+1), np.zeros(N+1), np.zeros(N+1)

theta_h[0], theta_r[0], theta_m[0] = 0.4, 0.3, 0.3
S_r[0], S_h[0], S_m[0], W[0] = 5, 5, 5, 15
for n, _ in enumerate(t):
    
    dz_r, dz_h = np.random.normal(loc=0, scale=np.sqrt(dt), size=2)

    #
    # S_r[n+1] = S_r[n] + mu_r*dt + sigma_r*dz_r
    # S_h[n+1] = S_h[n] + mu_h*dt + sigma_h*dz_h
    # S_m[n+1] = S_m[n]

    S_r[n] = (c_r*theta_r[n]*W[n])/p
    S_h[n] = (c_h*theta_h[n]*W[n])/p
    S_m[n] = (c_m*theta_m[n]*W[n])/p

    if S_r[n] > 1290:
        S_r[n] = 1290
    if S_h[n] > 0.22:
        S_h[n] = 0.22
    if S_m[n] > 150:
        S_m[n] = 150


    c_r = (K_r / S_r[n]) + ((OS_r*S_r[n]) / S_r[n])
    c_h = (K_h / S_h[n]) + ((OS_h*S_h[n]) / S_h[n])
    c_m = (K_m / S_m[n]) + ((OS_m*S_m[n]) / S_m[n])

    print("c_i :   ",c_r, c_h, c_m)

    W[n] = ((p/c_r)*S_r[n]) + ((p/c_h)*S_h[n]) + ((p/c_m)*S_m[n]) 
    
    print("W_n :   ", W[n])


    a_r = (mu_r / S_r[n]) - (mu_m / S_m[n])
    a_h = (mu_h / S_h[n]) - (mu_m / S_m[n])

    print("a_i :   ", a_r, a_h)

    theta_r[n] = ((mu_r / S_r[n]) - (mu_m / S_m[n])) * (S_r[n]**2 / (gamma * sigma_r**2))
    theta_h[n] = ((mu_h / S_h[n]) - (mu_m / S_m[n])) * (S_h[n]**2 / (gamma * sigma_h**2))
    theta_m[n] = 1 - theta_r[n] - theta_h[n]

    print("theta :   ", theta_r[n], theta_h[n], theta_m[n])

    x[n+1] = (
       W[n]
       *(
          ((delta - xi) / gamma) 
          - (
            ((1 - gamma) / gamma) 
              * (
                  (S_r[n]**2 / (2 * gamma * sigma_r**2)) * a_r**2 
                  + (S_h[n]**2 / (2 * gamma * sigma_h**2)) * a_h**2 
                  + (mu_m / S_m[n])
                )
            )
        )
    )

    print("x :   ", x)
    

    W[n+1] = (
        W[n] 
        + 
        (a_r * theta_r[n] + a_h * theta_h[n] + (mu_m / S_m[n])) * W[n]* dt
        - 
        x[n+1]* dt 
        + ((sigma_r / S_r[n])) * theta_r[n] * W[n] * dz_r 
        + ((sigma_h / S_h[n])) * theta_h[n] * W[n] * dz_h
    )

    print("W_n1 :   ", W[n+1])

    print("S_in1 :   ", S_r[n], S_h[n], S_m[n])

    print(f"End iteration {n}\n")


## Results Visualization & Behaver Analysis


In [ ]:
plt.plot(W)

In [ ]:
theta_h

In [ ]:
W[2] = ((p/c_r)*S_r[2]) + ((p/c_h)*S_h[2]) + ((p/c_m)*S_m[2]) 


In [ ]:
theta_r = (p*S_r[2])/(c_r*W[2])

theta_h = (p*S_h[2])/(c_h*W[2])

In [ ]:
theta_r + theta_h

In [ ]:
theta_h